In [1]:
import torch
import torchvision
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore') 

1a. 
weight: $ 5 \times 1 \times 4 \times 4 = 80$  
bias: 5

we have 5 channels in the feature maps. Each kernel has 4x4 dimension. We only have one channel in the input layer. 

1b.
the dimension of each feature map is 13 by 13.
since the input tensor's dimension was  $28 \times 28$ and the stride was 2, the kernel will start scanning at index 0 and end at index 14 with a stride of 2. There are a total of 13 scans. 

In [8]:
#2-a

## Create random tensor to represent a 7x7 image with 3 channels
random_tensor = torch.rand(1,3,7,7)

## Use random_tensor as input into Conv2d
from torch import nn
trial_net = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1)
trial_output = trial_net(random_tensor)
trial_relu = nn.ReLU()
trial_relu_result = trial_relu(trial_output)

trial_pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
trial_relu_pool = trial_pool(trial_relu_result)

trial_pool_result = trial_pool(trial_output)
trial_pool_relu = trial_relu(trial_pool_result)

print(f'relu folloed by pool: {trial_relu_pool}')
print(f'pool folloed by relu: {trial_pool_relu}')
(trial_relu_pool == trial_pool_relu).unique()


relu folloed by pool: tensor([[[[0.0688, 0.0274],
          [0.0000, 0.1710]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.3414, 0.4602],
          [0.0498, 0.1573]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)
pool folloed by relu: tensor([[[[0.0688, 0.0274],
          [0.0000, 0.1710]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.3414, 0.4602],
          [0.0498, 0.1573]]]], grad_fn=<ReluBackward0>)


tensor([True])

The resulting feature map is independent of the order of max pooling and relu activation. 

In [9]:
#2-b
trial_relu = nn.ReLU()
trial_relu_result = trial_relu(trial_output)

trial_pool = nn.AvgPool2d(kernel_size = 2, stride = 2)
trial_relu_pool = trial_pool(trial_relu_result)

trial_pool_result = trial_pool(trial_output)
trial_pool_relu = trial_relu(trial_pool_result)

print(f'relu folloed by pool: {trial_relu_pool}')
print(f'pool folloed by relu: {trial_pool_relu}')
(trial_relu_pool == trial_pool_relu).unique()

relu folloed by pool: tensor([[[[0.0172, 0.0069],
          [0.0000, 0.0910]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.1316, 0.1158],
          [0.0202, 0.0543]]]], grad_fn=<AvgPool2DBackward0>)
pool folloed by relu: tensor([[[[0.0000, 0.0000],
          [0.0000, 0.0522]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0493, 0.1114],
          [0.0000, 0.0000]]]], grad_fn=<ReluBackward0>)


tensor([False,  True])

these result are not same with the result we observedin part A when we use average pooling. 

3-1

the dimension of the output tensor produced is [1, 10, 25, 25]

the dimenion after  nn.ReLU(): [1, 10, 25, 25]

the dimension after nn.MaxPool2d(2,2): [1, 10, 12, 12]

3-2 

No, because the linear layer has a weight that takes in the dimension 750. Only when we have an image with dimension 28x28, can we pass that layer of network. 


3-3 

the dimensions of the output for each layer is labeled 

3-4 

We can change 250 without requiring a new format of images. 



In [11]:

from torch import nn

class my_net(nn.Module):
    
    ## Constructor commands
    def __init__(self):
        super(my_net, self).__init__()
        
        ## Define architecture
        self.conv_stack = nn.Sequential( # [1, 1, 28, 28]]
            nn.Conv2d(1, 10, 4, 1), # [1, 10, 25, 25]
            nn.ReLU(), # [1, 10, 25, 25]
            nn.MaxPool2d(2, 2), # [1, 10, 12, 12]
            nn.Conv2d(10, 30, 2, 1), # [1, 30, 10, 10]
            nn.ReLU(), # [1, 30, 10, 10]
            nn.MaxPool2d(2, 2), # [1, 30, 5, 5]
            nn.Flatten(), # [750]
            nn.Linear(750, 250),
            nn.ReLU(),
            nn.Linear(250, 10)
        )
    
    ## Function to generate predictions
    def forward(self, x):
        scores = self.conv_stack(x)
        return scores